In [1]:
import pandas as pd
import csv
import matplotlib.pyplot as plt
import seaborn as sns
import logging

import gradio as gr

#regression
from sklearn.linear_model import LinearRegression, Ridge, BayesianRidge, HuberRegressor, Lasso
from sklearn.metrics import make_scorer, mean_squared_error,confusion_matrix,accuracy_score
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, AdaBoostRegressor
from sklearn.svm import SVR  
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV 

#Classification
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression

ModuleNotFoundError: No module named 'seaborn'

In [ ]:
#logger
logging.basicConfig(filename='logs.log', level=logging.INFO,format='%(asctime)s :: %(levelname)s :: %(message)s')

### Data Exploration

In [ ]:
#reading and labeling data
def DataReading(filename):
    data=pd.read_csv(filename, sep=" ", header=None)
    # columns list
    COLS = ["ID", "Cycle", "OpSet1", "OpSet2", "OpSet3", "SensorMeasure1", "SensorMeasure2", "SensorMeasure3", "SensorMeasure4",
                "SensorMeasure5", "SensorMeasure6", "SensorMeasure7", "SensorMeasure8", "SensorMeasure9", "SensorMeasure10", "SensorMeasure11",
                "SensorMeasure12", "SensorMeasure13", "SensorMeasure14", "SensorMeasure15", "SensorMeasure16",
                "SensorMeasure17", "SensorMeasure18", "SensorMeasure19", "SensorMeasure20", "SensorMeasure21"]
    data =data[[f for f in range(0, 26)]]
    data.columns = ["ID", "Cycle", "OpSet1", "OpSet2", "OpSet3", "SensorMeasure1", "SensorMeasure2", "SensorMeasure3", "SensorMeasure4",
                "SensorMeasure5", "SensorMeasure6", "SensorMeasure7", "SensorMeasure8", "SensorMeasure9", "SensorMeasure10", "SensorMeasure11",
                "SensorMeasure12", "SensorMeasure13", "SensorMeasure14", "SensorMeasure15", "SensorMeasure16",
                "SensorMeasure17", "SensorMeasure18", "SensorMeasure19", "SensorMeasure20", "SensorMeasure21"]
    return data
    



#train data
train_data=DataReading("data set/aero/train_FD001.txt")

#test data
test_data=DataReading("data set/aero/test_FD001.txt")

logging.info("Training and Test Data Collection and labeling complete")


In [ ]:
# reading Rul of last cycle of each engin for test data
#reading data train data

Rul_data=pd.read_csv("data set/aero/RUL_FD001.txt", sep=" ", header=None)
Rul_data["ID"] = Rul_data.index + 1
Rul_data=Rul_data.rename(columns={0:"RUL"}).drop(1,axis=1)
Rul_data
logging.info("Remaining Usefull life of Test Data set reading/Loading complete")

THIS THE RUL OF LAST CYCLE OF DIFFRENT ENGINS OF TEST DATA

### Preprocessing

In [ ]:
# maximun number of engines cycle in train data
max_cycles_df_train = train_data.groupby(["ID"], sort=False)["Cycle"].max().reset_index().rename(columns={"Cycle" : "MaxCycleID"})
max_cycles_df_train



# Calculating RUL form the max_cycles_df in train data
train_data = pd.merge(train_data, max_cycles_df_train, how="inner", on="ID")
train_data["RUL"] = train_data["MaxCycleID"] - train_data["Cycle"]
train_data.drop('MaxCycleID',inplace=True, axis=1)
train_data.head()

#coverting into binary Rul 
# 1 for okay
# 0 for not Okay
binary_RUL=train_data['RUL'].map(lambda x: 1 if x>=20 else 0)
train_data['Binary_RUL']=binary_RUL
pro_train_data=train_data
pro_train_data
logging.info("Training Data Set RUL Calculation and Binary coversion of RUL complete")

In [ ]:
# maximun number of engines cycle in test data
max_cycles_df_test = test_data.copy(deep=True).groupby(['ID'])["Cycle"].max().reset_index()
max_cycles_df_test

test_data = pd.merge(test_data, max_cycles_df_test, how="inner", on=["ID",'Cycle'])
test_data["RUL"]=Rul_data["RUL"]

#coverting into binary Rul 
# 1 for okay
# 0 for not Okay
binary_RUL=test_data['RUL'].map(lambda x: 1 if x>=20 else 0)
test_data['Binary_RUL']=binary_RUL
pro_test_data=test_data
logging.info("Testing Data Set Binary coversion of RUL complete")
pro_test_data

In [ ]:
pd.set_option('display.max_columns', None)
pro_test_data

From the above graphs we can see that opset3 ,SensorMeasure 1,5,6,10,16,18,19 are constant for diffrent RULs.
so these features will not be use full for the training.